In [53]:
import numpy as np 
import pandas as pd 
import torch
import torch.nn as nn
import torch.nn.functional as Func

import matplotlib.pyplot as plt

from tqdm import tqdm

In [54]:
torch.cuda.is_available()
device = 'cuda' if torch.cuda.is_available() else 'cpu'
print(device)

cpu


In [55]:
path='./Iris.csv'
fullData=pd.read_csv(path)
fullData=fullData.sample(frac=1)
fullData[0:4]

# dataloader=torch.utils.data.DataLoader(dataset, batch_size=16,shuffle=True, num_workers = 4)

,Id,SepalLengthCm,SepalWidthCm,PetalLengthCm,PetalWidthCm,Species
102,103,7.1,3.0,5.9,2.1,Iris-virginica
105,106,7.6,3.0,6.6,2.1,Iris-virginica
89,90,5.5,2.5,4.0,1.3,Iris-versicolor
54,55,6.5,2.8,4.6,1.5,Iris-versicolor


In [56]:
fullData=fullData.drop(columns='Id')
clases=fullData.iloc[:,4].unique()
print(clases)

['Iris-virginica' 'Iris-versicolor' 'Iris-setosa']


In [57]:
newLabel={
    'Iris-setosa':0,
    'Iris-versicolor':1,
    'Iris-virginica':2
}
fullData=fullData.replace(newLabel)

In [58]:
fullData

,SepalLengthCm,SepalWidthCm,PetalLengthCm,PetalWidthCm,Species
102,7.1,3.0,5.9,2.1,2
105,7.6,3.0,6.6,2.1,2
89,5.5,2.5,4.0,1.3,1
54,6.5,2.8,4.6,1.5,1
14,5.8,4.0,1.2,0.2,0
...,...,...,...,...,...
137,6.4,3.1,5.5,1.8,2
31,5.4,3.4,1.5,0.4,0
138,6.0,3.0,4.8,1.8,2
50,7.0,3.2,4.7,1.4,1


In [59]:

CountOfRows=fullData.shape[0]
testData=fullData[0:CountOfRows//3].reset_index(drop=True)
trainData=fullData[CountOfRows//3:CountOfRows].reset_index(drop=True)
trainData


,SepalLengthCm,SepalWidthCm,PetalLengthCm,PetalWidthCm,Species
0,5.1,3.5,1.4,0.2,0
1,5.7,2.8,4.1,1.3,1
2,7.3,2.9,6.3,1.8,2
3,6.1,3.0,4.6,1.4,1
4,7.9,3.8,6.4,2.0,2
...,...,...,...,...,...
95,6.4,3.1,5.5,1.8,2
96,5.4,3.4,1.5,0.4,0
97,6.0,3.0,4.8,1.8,2
98,7.0,3.2,4.7,1.4,1


In [60]:
def normalizeCollumns(in_file):
    for i in range(in_file.shape[1]):
        column=in_file[in_file.columns[i]]
        in_file[in_file.columns[i]]=(column-np.min(column))/(np.max(column)-np.min(column))
    return in_file

In [61]:
class MyDataset(torch.utils.data.Dataset):
    def __init__(self,csv_file):
        data=csv_file
        x=data.iloc[:,:4]
        x=normalizeCollumns(x)
        y=data.iloc[:,4]
        self.x_train=torch.Tensor(np.array(x))
        self.y_train=nn.functional.one_hot(torch.Tensor(np.array(y)).to(torch.int64),num_classes=3)
        
        
    def __len__(self):
        return len(self.y_train)
    def __getitem__(self,idx):
        return self.x_train[idx],self.y_train[idx]

In [62]:
trainDataset=MyDataset(trainData)
trainDataloader= torch.utils.data.DataLoader(trainDataset,batch_size=5)

In [63]:
class IrisClassifier(torch.nn.Module):
    def __init__(self,countOfParametrs=4):
        super().__init__()
        act_func=torch.nn.ReLU()
        linear_1 = torch.nn.Linear(countOfParametrs,4*countOfParametrs)
        linear_2= torch.nn.Linear(4*countOfParametrs,3)

        self.model = torch.nn.Sequential(
            linear_1,act_func,linear_2
        )

    def forward(self,x):
        return self.model(x)

In [64]:
model=IrisClassifier().to(device)
loss_func=torch.nn.CrossEntropyLoss()
optimiser=torch.optim.Adam(model.parameters(),lr=1e-2)

In [65]:
def accruracy(pred,label):
    answer = pred.detach().numpy().argmax(1) == label.numpy().argmax(1)
    
    return answer.mean()


In [66]:
def trainModel(dataloader,model,loss_fn,optimiser):
    loss_val=0
    accruracy_val=0
    for sample in tqdm(dataloader):
        X,Y=sample
        X=X.to(device)
        Y=Y.to(device)
        
        Y=Y.to(torch.float32)
        
        pred=model(X)
        
    
        loss=loss_fn(pred,Y)
        
        optimiser.zero_grad()
        loss.backward()
        optimiser.step()
        loss_val+=loss.item()
        accruracy_val+= accruracy(pred.cpu(),Y.cpu())
        
    print("avg loss=",loss_val/len(dataloader))
    print("avg accuracy=",accruracy_val/len(dataloader))
    


In [67]:
countOfEpoch=15
for epoch in range(countOfEpoch):
    print("Epoch#",epoch+1)
    trainModel(trainDataloader,model,loss_func,optimiser)

Epoch# 1


100%|██████████| 20/20 [00:00<00:00, 410.80it/s]


avg loss= 0.9330822080373764
avg accuracy= 0.68
Epoch# 2


100%|██████████| 20/20 [00:00<00:00, 434.99it/s]


avg loss= 0.6720385313034057
avg accuracy= 0.8100000000000002
Epoch# 3


...

avg loss= 0.15370811633765696
avg accuracy= 0.9600000000000002
Epoch# 14


100%|██████████| 20/20 [00:00<00:00, 537.50it/s]


avg loss= 0.14363634446635842
avg accuracy= 0.9600000000000002
Epoch# 15


100%|██████████| 20/20 [00:00<00:00, 526.25it/s]

avg loss= 0.13509682863950728
avg accuracy= 0.9600000000000002


In [68]:

def testModel(dataloader,model):
    accruracy_val=0
    for sample in dataloader:
        X,Y=sample
        
        X=X.to(device)
        Y=Y.to(device)
        
        X=X.to(torch.float32)
        pred=model(X)
        accruracy_val+= accruracy(pred.cpu(),Y.cpu())
    print("accuracy=",accruracy_val/len(dataloader))

In [69]:
testDataset=MyDataset(testData)
testDataloader=torch.utils.data.DataLoader(testDataset,batch_size=1)
testModel(testDataloader,model)

accuracy= 0.98
